### Mounting drive on Colab

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


###Libraries

In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Effect of changing hidden layers



In order to study the effects of changing number of hidden layers totl number of neurons must be kept constant across all layers while training the model. Therefore, the following experimental setup is used here:


For Ex: 
        
        If number of neurons be = 1024

        number of hidden layers = 1

        
        If number of hidden layers = 2

        Number of neurons per hidden layer are reduced to 512

        
        Hence, Number of layers: [1, 2, 4, 8, 16, 32, 64]

        Number of Neurons per layer: [1024, 512, 256, 128, 64, 32, 16] 



### For ImageNet as a source

In [ ]:
def compile_model(num_neurons, num_layers):
  
  from timeit import default_timer as timer
  class TimingCallback(keras.callbacks.Callback):
      def __init__(self, logs={}):
          self.logs=[]
      def on_epoch_begin(self, epoch, logs={}):
          self.starttime = timer()
      def on_epoch_end(self, epoch, logs={}):
          self.logs.append(timer()-self.starttime)

  cb = TimingCallback()

  from tensorflow.keras.applications.inception_v3 import InceptionV3
  pre_trained_model = InceptionV3(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = 'None')
  #for layer in pre_trained_model.layers:
  #  layer.trainable = False

  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)

  for i in range(num_layers):
    # Add a fully connected layer with "num_neurons" hidden units and ReLU activation
    x = layers.Dense(num_neurons, activation='relu')(x)                  
  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model, cb

In [ ]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [ ]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [ ]:
print("*********** Fitting model for 1024 neurons size and 1 hidden layers")
model, cb = compile_model(1024, 1)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 1024 neurons size and 1 hidden layers
87924736/87910968 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 30s 60ms/step - loss: 24.1073 - acc: 0.1236 - auc: 0.5466 - recall: 0.1191 - precision: 0.1257 - hamming_loss: 0.8809 - f1_score: 0.1223 - val_loss: 13.8997 - val_acc: 0.1470 - val_auc: 0.5697 - val_recall: 0.1393 - val_precision: 0.1486 - val_hamming_loss: 0.8607 - val_f1_score: 0.1152
Epoch 2/10
200/200 [==============================] - 10s 51ms/step - loss: 16.1683 - acc: 0.1709 - auc: 0.5852 - recall: 0.1629 - precision: 0.1754 - hamming_loss: 0.8371 - f1_score: 0.1688 - val_loss: 17.3577 - val_acc: 0.1437 - val_auc: 0.5683 - val_recall: 0.1400 - val_precision: 0.1471 - val_hamming_loss: 0.8600 - val_f1_score: 0.1168
Epoch 3/10
200/200 [==============================] - 10s 51ms/step - loss: 13.0374 - acc: 0.2033 - auc: 0.6089 - recall: 0.1914 - precision: 0.2097 - hamming_loss: 0.8086 - f1_score: 0.1998 - val_loss: 12.3234 - val_acc: 0.2240 - val_auc: 0.6201 - val_recall: 0.2120 - val_precision: 0.2376 - val_hamming_loss: 0.78

In [ ]:
print("*********** Fitting model for 512 dataset size and 2 hidden layers")
model, cb = compile_model(512, 2)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 512 dataset size and 2 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 18s 61ms/step - loss: 12.4618 - acc: 0.1295 - auc_1: 0.5588 - recall_1: 0.1123 - precision_1: 0.1321 - hamming_loss: 0.8877 - f1_score: 0.1211 - val_loss: 7.1850 - val_acc: 0.1583 - val_auc_1: 0.6014 - val_recall_1: 0.1290 - val_precision_1: 0.1771 - val_hamming_loss: 0.8710 - val_f1_score: 0.1169
Epoch 2/10
200/200 [==============================] - 11s 54ms/step - loss: 6.1773 - acc: 0.1773 - auc_1: 0.6187 - recall_1: 0.1419 - precision_1: 0.1941 - hamming_loss: 0.8581 - f1_score: 0.1634 - val_loss: 4.7471 - val_acc: 0.2177 - val_auc_1: 0.6536 - val_recall_1: 0.1700 - val_precision_1: 0.2434 - val_hamming_loss: 0.8300 - val_f1_score: 0.1754
Epoch 3/10
200/200 [==============================] - 11s 53ms/step - loss: 4.3335 - acc: 0.2243 - auc_1: 0.6731 - recall_1: 0.1706 - precision_1: 0.2673 - hamming_loss: 0.8294 - f1_score: 0.2068 - val_loss: 3.8846 - val_acc: 0.2327 - val_auc_1: 0.6818 - val_recall_1: 0.1670 - val_precision_1: 

In [ ]:
print("*********** Fitting model for 256 dataset size and 4 hidden layers")
model, cb = compile_model(256, 4)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 256 dataset size and 4 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 17s 62ms/step - loss: 5.0816 - acc: 0.1280 - auc_2: 0.5820 - recall_2: 0.0635 - precision_2: 0.1416 - hamming_loss: 0.9365 - f1_score: 0.0868 - val_loss: 2.9854 - val_acc: 0.1743 - val_auc_2: 0.6556 - val_recall_2: 0.0587 - val_precision_2: 0.2785 - val_hamming_loss: 0.9413 - val_f1_score: 0.0890
Epoch 2/10
200/200 [==============================] - 11s 53ms/step - loss: 2.8623 - acc: 0.1961 - auc_2: 0.6802 - recall_2: 0.0735 - precision_2: 0.3209 - hamming_loss: 0.9265 - f1_score: 0.1163 - val_loss: 3.0074 - val_acc: 0.1720 - val_auc_2: 0.6714 - val_recall_2: 0.0780 - val_precision_2: 0.2505 - val_hamming_loss: 0.9220 - val_f1_score: 0.0862
Epoch 3/10
200/200 [==============================] - 11s 55ms/step - loss: 2.4660 - acc: 0.2519 - auc_2: 0.7394 - recall_2: 0.0897 - precision_2: 0.4615 - hamming_loss: 0.9103 - f1_score: 0.1451 - val_loss: 2.3429 - val_acc: 0.2947 - val_auc_2: 0.7598 - val_recall_2: 0.0947 - val_precision_2: 0

In [ ]:
print("*********** Fitting model for 128 dataset size and 8 hidden layers")
model, cb = compile_model(128, 8)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 128 dataset size and 8 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 19s 68ms/step - loss: 2.7237 - acc: 0.1366 - auc_3: 0.6136 - recall_3: 0.0072 - precision_3: 0.3245 - hamming_loss: 0.9928 - f1_score: 0.0133 - val_loss: 2.5278 - val_acc: 0.1800 - val_auc_3: 0.6786 - val_recall_3: 0.0083 - val_precision_3: 0.5319 - val_hamming_loss: 0.9917 - val_f1_score: 0.0148
Epoch 2/10
200/200 [==============================] - 11s 56ms/step - loss: 2.4419 - acc: 0.2039 - auc_3: 0.7097 - recall_3: 0.0208 - precision_3: 0.6553 - hamming_loss: 0.9792 - f1_score: 0.0361 - val_loss: 2.4225 - val_acc: 0.2097 - val_auc_3: 0.7222 - val_recall_3: 0.0160 - val_precision_3: 0.5714 - val_hamming_loss: 0.9840 - val_f1_score: 0.0285
Epoch 3/10
200/200 [==============================] - 11s 57ms/step - loss: 2.3137 - acc: 0.2490 - auc_3: 0.7533 - recall_3: 0.0387 - precision_3: 0.6974 - hamming_loss: 0.9613 - f1_score: 0.0665 - val_loss: 2.3060 - val_acc: 0.2647 - val_auc_3: 0.7550 - val_recall_3: 0.0457 - val_precision_3: 0

In [ ]:
print("*********** Fitting model for 64 dataset size and 16 hidden layers")
model, cb = compile_model(64, 16)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 64 dataset size and 16 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 20s 67ms/step - loss: 2.6952 - acc: 0.0867 - auc_4: 0.5444 - recall_4: 0.0000e+00 - precision_4: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.6684 - val_acc: 0.0987 - val_auc_4: 0.5885 - val_recall_4: 0.0000e+00 - val_precision_4: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 11s 57ms/step - loss: 2.6247 - acc: 0.1218 - auc_4: 0.6263 - recall_4: 0.0000e+00 - precision_4: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.5792 - val_acc: 0.1500 - val_auc_4: 0.6556 - val_recall_4: 0.0000e+00 - val_precision_4: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
200/200 [==============================] - 11s 56ms/step - loss: 2.5300 - acc: 0.1598 - auc_4: 0.6804 - recall_4: 4.5833e-04 - precision_4: 0.6111 - hamming_loss: 0.9995 - f1_score: 9.0651e-04 - val_loss: 2.4726 - val_acc: 0.1853 - val_a

In [ ]:
print("*********** Fitting model for 32 dataset size and 32 hidden layers")
model, cb = compile_model(32, 32)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 32 dataset size and 32 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 24s 76ms/step - loss: 2.7058 - acc: 0.0817 - auc_5: 0.5100 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7011 - val_acc: 0.0880 - val_auc_5: 0.5222 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 12s 59ms/step - loss: 2.6896 - acc: 0.1020 - auc_5: 0.5529 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.6802 - val_acc: 0.1057 - val_auc_5: 0.5779 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
200/200 [==============================] - 12s 59ms/step - loss: 2.6636 - acc: 0.1072 - auc_5: 0.5962 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.6529 - val_acc: 0.1030 - v

In [ ]:
print("*********** Fitting model for 16 dataset size and 64 hidden layers")
model, cb = compile_model(16, 64)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 16 dataset size and 64 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 27s 75ms/step - loss: 2.7081 - acc: 0.0642 - auc_7: 0.5000 - recall_7: 0.0000e+00 - precision_7: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7080 - val_acc: 0.0673 - val_auc_7: 0.5000 - val_recall_7: 0.0000e+00 - val_precision_7: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 12s 61ms/step - loss: 2.7081 - acc: 0.0644 - auc_7: 0.5000 - recall_7: 0.0000e+00 - precision_7: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7080 - val_acc: 0.0673 - val_auc_7: 0.5000 - val_recall_7: 0.0000e+00 - val_precision_7: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
200/200 [==============================] - 12s 61ms/step - loss: 2.7081 - acc: 0.0635 - auc_7: 0.5000 - recall_7: 0.0000e+00 - precision_7: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7080 - val_acc: 0.0673 - v